In [1]:
#IMPORTING DEPENDENCIES
import pandas as pd
import _pickle as cPickle
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D
from tensorflow.keras.layers import MaxPooling1D, Embedding, LSTM,Bidirectional,Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import Constant
from keras.models import model_from_json
from keras.models import Model
import pickle





In [2]:
#we have two dataframes working below: data1(smaller cleaned data) and final_df(bigger data)
#The embeddings that we have perfectly matches the sequence of products in final_df 
#Thus first embeddings are populated in final_df and then we map asin ID of product to embeddings
#Finally we use this mapping to populate product embeddings in data1
data1=pd.read_excel("cleaned_data.xlsx")

In [3]:
data1.head()

,category,description,title,also_buy,brand,rank,also_view,main_cat,price,asin,tech1,image,date,feature,similar_item,details,tech2
0,"['Appliances', 'Parts & Accessories', 'Range P...",Outfit your dryer with top-notch connections w...,"Leviton 5050 B01-0-000 Electrical Receptacle, ...","B000FP8LK6, B00002N7IL, B000FPDO4Y, B00NPZH820...",Leviton,">#155,450 in Tools & Home Improvement (See top...",NaN,Tools & Home Improvement,$6.98,B00002N7HY,"{'Part Number': 'B01-05050-000', 'Item Weight'...",NaN,"November 9, 1999","['N/A', 'Made in USA', 'Built of durable therm...","[{'asin': '', 'href': '', 'name': 'Leviton 505...",NaN,NaN
1,"['Appliances', 'Parts & Accessories', 'Range P...",Outfit your dryer with top-notch connections w...,"Leviton 5206 50 Amp, 125/250 Volt, NEMA 10-50R...","B000FP8LK6, B00002N7HY, B000GAQG5C, B004LYLP7I...",Leviton,">#411,187 in Tools & Home Improvement (See top...","['B00002N7HY', 'B074Q2KZGV', 'B000FPCL2K', 'B0...",Tools & Home Improvement,NaN,B00002N7IL,"{'Part Number': 'B01-5206-000', 'Item Weight':...",NaN,"July 7, 2004","['Fits all wallplates with 2.15"" diameter cent...","[{'asin': '', 'href': '', 'name': 'Leviton 520...",NaN,NaN
2,"['Appliances', 'Parts & Accessories', 'Dryer P...",White heat keeper kitProFlex heat keeper kit. ...,"Dundas Jafine CHK100ZW CHK100ZW6 Vents, 4-Inch...","B0015UGPWQ, B001AAEG5O, B000BO76WU, B00009W3HD...",Dundas Jafine,">#78,624 in Tools & Home Improvement (See top ...",NaN,Tools & Home Improvement,$9.90,B00004YWK2,"{'Part Number': 'CHK100ZW', 'Item Weight': '10...",NaN,"June 16, 2006","['Directs warm, moist air from electric clothe...","[{'asin': '', 'href': '', 'name': 'Dundas Jafi...",NaN,NaN
3,"['Appliances', 'Parts & Accessories']",Keep your air humidifier operating at peak eff...,"Holmes ""C"" Humidifier Filter, HWF65PDQ-U","B0002TSA8I, B0002GD194, B0175Y17SW, B01G6921ZU",Holmes,">#1,235 in Home & Kitchen (See Top 100 in Home...",NaN,Amazon Home,$5.98,B00005OU6T,NaN,['https://images-na.ssl-images-amazon.com/imag...,NaN,"['Improves efficiency of the humidifier', 'Ext...","[{'asin': '', 'href': '', 'name': 'Holmes ""C"" ...",NaN,NaN
4,"['Appliances', 'Parts & Accessories']","Holmes ""A"" Humidifier Filter, HWF62","Holmes ""A"" Humidifier Filter, HWF62",B0002TSA8I,Holmes,">#1,074 in Home & Kitchen (See Top 100 in Home...",NaN,Amazon Home,$11.89,B00006IV17,NaN,['https://images-na.ssl-images-amazon.com/imag...,NaN,"['Nullify', 'Imported', 'Fits Holmes, Sunbeam,...","[{'asin': '', 'href': '', 'name': 'Holmes ""A"" ...",NaN,NaN


In [4]:
final_df=pd.read_csv("final_df.csv")

In [5]:
final_df.head()

,Unnamed: 0,index,category,description,title,also_buy,brand,rank,also_view,main_cat,price,asin,tech1,image,date,feature,similar_item,details,tech2
0,2,2,"['Appliances', 'Parts & Accessories']",This is an authorized aftermarket product. Fit...,Panasonic ADD96A1052 Blade,NaN,Panasonic,">#1,465,449 in Tools & Home Improvement (See t...",NaN,Tools & Home Improvement,NaN,6040985461,"{'Part Number': 'MSCADD96A1054', 'Item Weight'...",['https://images-na.ssl-images-amazon.com/imag...,"December 26, 2014","['This is an O.E.M. Authorized part', 'Fits wi...",NaN,NaN,NaN
1,5,7,"['Appliances', 'Parts & Accessories', 'Range P...",Full gauge and size beveled-edge; furnished wi...,EATON Wiring 39CH-SP-L Arrow Hart 1-Gang Chrom...,NaN,EATON Wiring,">#3,066,990 in Tools & Home Improvement (See t...",NaN,Tools & Home Improvement,$3.43,B00002N5EL,"{'Part Number': '39CH-SP-L', 'Item Weight': '1...",NaN,"January 16, 2007",['Returns will not be honored on this closeout...,NaN,NaN,NaN
2,6,8,"['Appliances', 'Parts & Accessories', 'Range P...",Outfit your dryer with top-notch connections w...,"Leviton 5050 B01-0-000 Electrical Receptacle, ...","['B000BPFZHM', 'B000FP8LK6', 'B07D5RQ5YL', 'B0...",Leviton,">#155,450 in Tools & Home Improvement (See top...",NaN,Tools & Home Improvement,$6.98,B00002N7HY,"{'Part Number': 'B01-05050-000', 'Item Weight'...",NaN,"November 9, 1999","['N/A', 'Made in USA', 'Built of durable therm...","[{'asin': '', 'href': '', 'name': 'Leviton 505...",NaN,NaN
3,7,9,"['Appliances', 'Parts & Accessories', 'Range P...",Outfit your dryer with top-notch connections w...,"Leviton 5206 50 Amp, 125/250 Volt, NEMA 10-50R...","['B000FK9X6W', 'B000BPFZHM', 'B000FP8LK6', 'B0...",Leviton,">#411,187 in Tools & Home Improvement (See top...","['B00002N7HY', 'B074Q2KZGV', 'B000FPCL2K', 'B0...",Tools & Home Improvement,NaN,B00002N7IL,"{'Part Number': 'B01-5206-000', 'Item Weight':...",NaN,"July 7, 2004","['Fits all wallplates with 2.15"" diameter cent...","[{'asin': '', 'href': '', 'name': 'Leviton 520...",NaN,NaN
4,9,11,"['Appliances', 'Parts & Accessories', 'Range P...",Flush dryer outlet will accept aluminum or cop...,Leviton 5207 125/250V Flush Mount Receptacle,NaN,Leviton,">#628,182 in Tools & Home Improvement (See Top...",NaN,Tools & Home Improvement,$5.79,B00002NARC,NaN,NaN,NaN,"['N/A', '3 wire', 'UL listed', 'Great for all ...","[{'asin': '', 'href': '', 'name': 'Leviton 520...",NaN,NaN


In [6]:
final_df.shape

(25327, 19)

In [7]:
#unpickiling embedding 50-d
def unpickle(filename):
    f = open(filename, "rb")
    d = cPickle.load(f)
    f.close()
    return d


In [8]:
data=unpickle("pca_50d_embeddings.pkl")

In [9]:
data.shape

(25327, 50)

In [10]:
type(data)

numpy.ndarray

In [11]:
#convert numpy array into pandas dataframe
data_df=pd.DataFrame(data)

In [12]:
#adding column of embedding
final_df['embedding']= data_df.values.tolist()

In [13]:
final_df

,Unnamed: 0,index,category,description,title,also_buy,brand,rank,also_view,main_cat,price,asin,tech1,image,date,feature,similar_item,details,tech2,embedding
0,2,2,"['Appliances', 'Parts & Accessories']",This is an authorized aftermarket product. Fit...,Panasonic ADD96A1052 Blade,NaN,Panasonic,">#1,465,449 in Tools & Home Improvement (See t...",NaN,Tools & Home Improvement,NaN,6040985461,"{'Part Number': 'MSCADD96A1054', 'Item Weight'...",['https://images-na.ssl-images-amazon.com/imag...,"December 26, 2014","['This is an O.E.M. Authorized part', 'Fits wi...",NaN,NaN,NaN,"[-1.1160051822662354, 0.12084303796291351, -0...."
1,5,7,"['Appliances', 'Parts & Accessories', 'Range P...",Full gauge and size beveled-edge; furnished wi...,EATON Wiring 39CH-SP-L Arrow Hart 1-Gang Chrom...,NaN,EATON Wiring,">#3,066,990 in Tools & Home Improvement (See t...",NaN,Tools & Home Improvement,$3.43,B00002N5EL,"{'Part Number': '39CH-SP-L', 'Item Weight': '1...",NaN,"January 16, 2007",['Returns will not be honored on this closeout...,NaN,NaN,NaN,"[0.13942743837833405, -0.4813501536846161, -0...."
2,6,8,"['Appliances', 'Parts & Accessories', 'Range P...",Outfit your dryer with top-notch connections w...,"Leviton 5050 B01-0-000 Electrical Receptacle, ...","['B000BPFZHM', 'B000FP8LK6', 'B07D5RQ5YL', 'B0...",Leviton,">#155,450 in Tools & Home Improvement (See top...",NaN,Tools & Home Improvement,$6.98,B00002N7HY,"{'Part Number': 'B01-05050-000', 'Item Weight'...",NaN,"November 9, 1999","['N/A', 'Made in USA', 'Built of durable therm...","[{'asin': '', 'href': '', 'name': 'Leviton 505...",NaN,NaN,"[1.527734398841858, -0.3872746229171753, 0.390..."
3,7,9,"['Appliances', 'Parts & Accessories', 'Range P...",Outfit your dryer with top-notch connections w...,"Leviton 5206 50 Amp, 125/250 Volt, NEMA 10-50R...","['B000FK9X6W', 'B000BPFZHM', 'B000FP8LK6', 'B0...",Leviton,">#411,187 in Tools & Home Improvement (See top...","['B00002N7HY', 'B074Q2KZGV', 'B000FPCL2K', 'B0...",Tools & Home Improvement,NaN,B00002N7IL,"{'Part Number': 'B01-5206-000', 'Item Weight':...",NaN,"July 7, 2004","['Fits all wallplates with 2.15"" diameter cent...","[{'asin': '', 'href': '', 'name': 'Leviton 520...",NaN,NaN,"[1.8896135091781616, -0.14123277366161346, 0.4..."
4,9,11,"['Appliances', 'Parts & Accessories', 'Range P...",Flush dryer outlet will accept aluminum or cop...,Leviton 5207 125/250V Flush Mount Receptacle,NaN,Leviton,">#628,182 in Tools & Home Improvement (See Top...",NaN,Tools & Home Improvement,$5.79,B00002NARC,NaN,NaN,NaN,"['N/A', '3 wire', 'UL listed', 'Great for all ...","[{'asin': '', 'href': '', 'name': 'Leviton 520...",NaN,NaN,"[2.166203498840332, -0.04293861612677574, 0.48..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25322,28148,30454,"['Appliances', 'Parts & Accessories']",This is an authorized aftermarket product. Fit...,Bosch 00175338 Switch,NaN,Bosch,">#1,565,533 in Tools & Home Improvement (See t...",NaN,Tools & Home Improvement,$40.23,B01HJH92JQ,"{'Part Number': '00175338', 'Item Weight': '1....",NaN,"June 25, 2016","['This is an O.E.M. Authorized part', 'Fits wi...",NaN,NaN,NaN,"[-1.2052078247070312, 0.17841953039169312, -0...."
25323,28149,30455,"['Appliances', 'Parts & Accessories']",This is an authorized aftermarket product. Fit...,Bosch 00478807 Panel-Facia,NaN,Bosch,">#640,724 in Tools & Home Improvement (See top...","['B004Q3XUJA', 'B004T0GLW8', 'B00K433QFY', 'B0...",Tools & Home Improvement,$95.99,B01HJH2I5G,"{'Part Number': '00478807', 'Item Weight': '1....",['https://images-na.ssl-images-amazon.com/imag...,"March 3, 2010","['This is an O.E.M. Authorized part', 'Fits wi...",NaN,NaN,NaN,"[-1.2052078247070312, 0.17841953039169312, -0...."
25324,28150,30456,"['Appliances', 'Parts & Accessories']",This is an authorized aftermarket product. Fit...,Bosch 00649288 Ice Maker,NaN,Bosch,">#759,045 in Tools & Home Improvement (See top...","['B004XL2U1O', 'B002SZPM5I', 'B01HJH0J4S']",Tools & Home Improvement,$250.79,B01

In [14]:
#dictionary for products
enc=dict(zip(final_df["asin"],final_df["embedding"]))

In [15]:
#mapping in dataframe
data1["embedding"]=data1["asin"].map(enc)

In [ ]:
#mapping procedure is now finished

In [16]:
data1.head()

,category,description,title,also_buy,brand,rank,also_view,main_cat,price,asin,tech1,image,date,feature,similar_item,details,tech2,embedding
0,"['Appliances', 'Parts & Accessories', 'Range P...",Outfit your dryer with top-notch connections w...,"Leviton 5050 B01-0-000 Electrical Receptacle, ...","B000FP8LK6, B00002N7IL, B000FPDO4Y, B00NPZH820...",Leviton,">#155,450 in Tools & Home Improvement (See top...",NaN,Tools & Home Improvement,$6.98,B00002N7HY,"{'Part Number': 'B01-05050-000', 'Item Weight'...",NaN,"November 9, 1999","['N/A', 'Made in USA', 'Built of durable therm...","[{'asin': '', 'href': '', 'name': 'Leviton 505...",NaN,NaN,"[1.527734398841858, -0.3872746229171753, 0.390..."
1,"['Appliances', 'Parts & Accessories', 'Range P...",Outfit your dryer with top-notch connections w...,"Leviton 5206 50 Amp, 125/250 Volt, NEMA 10-50R...","B000FP8LK6, B00002N7HY, B000GAQG5C, B004LYLP7I...",Leviton,">#411,187 in Tools & Home Improvement (See top...","['B00002N7HY', 'B074Q2KZGV', 'B000FPCL2K', 'B0...",Tools & Home Improvement,NaN,B00002N7IL,"{'Part Number': 'B01-5206-000', 'Item Weight':...",NaN,"July 7, 2004","['Fits all wallplates with 2.15"" diameter cent...","[{'asin': '', 'href': '', 'name': 'Leviton 520...",NaN,NaN,"[1.8896135091781616, -0.14123277366161346, 0.4..."
2,"['Appliances', 'Parts & Accessories', 'Dryer P...",White heat keeper kitProFlex heat keeper kit. ...,"Dundas Jafine CHK100ZW CHK100ZW6 Vents, 4-Inch...","B0015UGPWQ, B001AAEG5O, B000BO76WU, B00009W3HD...",Dundas Jafine,">#78,624 in Tools & Home Improvement (See top ...",NaN,Tools & Home Improvement,$9.90,B00004YWK2,"{'Part Number': 'CHK100ZW', 'Item Weight': '10...",NaN,"June 16, 2006","['Directs warm, moist air from electric clothe...","[{'asin': '', 'href': '', 'name': 'Dundas Jafi...",NaN,NaN,"[-0.30327674746513367, -0.3628358840942383, -0..."
3,"['Appliances', 'Parts & Accessories']",Keep your air humidifier operating at peak eff...,"Holmes ""C"" Humidifier Filter, HWF65PDQ-U","B0002TSA8I, B0002GD194, B0175Y17SW, B01G6921ZU",Holmes,">#1,235 in Home & Kitchen (See Top 100 in Home...",NaN,Amazon Home,$5.98,B00005OU6T,NaN,['https://images-na.ssl-images-amazon.com/imag...,NaN,"['Improves efficiency of the humidifier', 'Ext...","[{'asin': '', 'href': '', 'name': 'Holmes ""C"" ...",NaN,NaN,"[2.198042631149292, -0.004325718153268099, 0.3..."
4,"['Appliances', 'Parts & Accessories']","Holmes ""A"" Humidifier Filter, HWF62","Holmes ""A"" Humidifier Filter, HWF62",B0002TSA8I,Holmes,">#1,074 in Home & Kitchen (See Top 100 in Home...",NaN,Amazon Home,$11.89,B00006IV17,NaN,['https://images-na.ssl-images-amazon.com/imag...,NaN,"['Nullify', 'Imported', 'Fits Holmes, Sunbeam,...","[{'asin': '', 'href': '', 'name': 'Holmes ""A"" ...",NaN,NaN,"[-1.6430426836013794, 0.4439624845981598, 0.28..."


In [ ]:
data1.isnull().sum()

In [17]:
#Saving DataFrame used as input to LSTM
data1.to_csv("lstm_input.csv")

In [18]:
#DATA CLEANING
#dropping all the nan values from also_buy
data_new=data1.dropna(subset=["also_buy"])

In [19]:
data_new["also_buy"][0]

'B000FP8LK6, B00002N7IL, B000FPDO4Y, B00NPZH820, B000BQ8D7A, B00FI6U0GU'

In [20]:
data_new.head()

,category,description,title,also_buy,brand,rank,also_view,main_cat,price,asin,tech1,image,date,feature,similar_item,details,tech2,embedding
0,"['Appliances', 'Parts & Accessories', 'Range P...",Outfit your dryer with top-notch connections w...,"Leviton 5050 B01-0-000 Electrical Receptacle, ...","B000FP8LK6, B00002N7IL, B000FPDO4Y, B00NPZH820...",Leviton,">#155,450 in Tools & Home Improvement (See top...",NaN,Tools & Home Improvement,$6.98,B00002N7HY,"{'Part Number': 'B01-05050-000', 'Item Weight'...",NaN,"November 9, 1999","['N/A', 'Made in USA', 'Built of durable therm...","[{'asin': '', 'href': '', 'name': 'Leviton 505...",NaN,NaN,"[1.527734398841858, -0.3872746229171753, 0.390..."
1,"['Appliances', 'Parts & Accessories', 'Range P...",Outfit your dryer with top-notch connections w...,"Leviton 5206 50 Amp, 125/250 Volt, NEMA 10-50R...","B000FP8LK6, B00002N7HY, B000GAQG5C, B004LYLP7I...",Leviton,">#411,187 in Tools & Home Improvement (See top...","['B00002N7HY', 'B074Q2KZGV', 'B000FPCL2K', 'B0...",Tools & Home Improvement,NaN,B00002N7IL,"{'Part Number': 'B01-5206-000', 'Item Weight':...",NaN,"July 7, 2004","['Fits all wallplates with 2.15"" diameter cent...","[{'asin': '', 'href': '', 'name': 'Leviton 520...",NaN,NaN,"[1.8896135091781616, -0.14123277366161346, 0.4..."
2,"['Appliances', 'Parts & Accessories', 'Dryer P...",White heat keeper kitProFlex heat keeper kit. ...,"Dundas Jafine CHK100ZW CHK100ZW6 Vents, 4-Inch...","B0015UGPWQ, B001AAEG5O, B000BO76WU, B00009W3HD...",Dundas Jafine,">#78,624 in Tools & Home Improvement (See top ...",NaN,Tools & Home Improvement,$9.90,B00004YWK2,"{'Part Number': 'CHK100ZW', 'Item Weight': '10...",NaN,"June 16, 2006","['Directs warm, moist air from electric clothe...","[{'asin': '', 'href': '', 'name': 'Dundas Jafi...",NaN,NaN,"[-0.30327674746513367, -0.3628358840942383, -0..."
3,"['Appliances', 'Parts & Accessories']",Keep your air humidifier operating at peak eff...,"Holmes ""C"" Humidifier Filter, HWF65PDQ-U","B0002TSA8I, B0002GD194, B0175Y17SW, B01G6921ZU",Holmes,">#1,235 in Home & Kitchen (See Top 100 in Home...",NaN,Amazon Home,$5.98,B00005OU6T,NaN,['https://images-na.ssl-images-amazon.com/imag...,NaN,"['Improves efficiency of the humidifier', 'Ext...","[{'asin': '', 'href': '', 'name': 'Holmes ""C"" ...",NaN,NaN,"[2.198042631149292, -0.004325718153268099, 0.3..."
4,"['Appliances', 'Parts & Accessories']","Holmes ""A"" Humidifier Filter, HWF62","Holmes ""A"" Humidifier Filter, HWF62",B0002TSA8I,Holmes,">#1,074 in Home & Kitchen (See Top 100 in Home...",NaN,Amazon Home,$11.89,B00006IV17,NaN,['https://images-na.ssl-images-amazon.com/imag...,NaN,"['Nullify', 'Imported', 'Fits Holmes, Sunbeam,...","[{'asin': '', 'href': '', 'name': 'Holmes ""A"" ...",NaN,NaN,"[-1.6430426836013794, 0.4439624845981598, 0.28..."


In [21]:
#also_buy will be paded to 30 product ids
MAX_SEQUENCE_LENGTH = 30

In [22]:
#50 dimensional vector representation of product
EMBEDDING_DIM = 50
VALIDATION_SPLIT = 0.2

In [23]:
data_new.shape

(6040, 18)

In [24]:
#number of unique product ids
data_new["asin"].nunique()

6001

In [25]:
MAX_NUM_WORDS=6001

In [26]:
# creating list of array containing sequence of products
docs=data_new["also_buy"].tolist()

In [27]:
#Treating Product ID as word, Tokenize them 
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(docs)
sequences = tokenizer.texts_to_sequences(docs)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
#to create sequence of equal length
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', data.shape)


Found 5013 unique tokens.
Shape of data tensor: (6040, 30)


In [28]:
#convert label list into array
# preparing output for the REGRESSION TASK
labels_product=data_new["embedding"].to_numpy()


In [105]:
labels_product

array([[-0.62389219, -0.20638207, -0.35936728, ...,  0.02577148,
        -0.0012505 , -0.02772814],
       [-0.35737041, -0.26035231, -0.29370376, ...,  0.04674413,
        -0.00244801, -0.04100928],
       [-0.74219096, -0.16410044, -0.40034229, ..., -0.00370413,
         0.02244856,  0.01156897],
       ...,
       [-1.0389576 ,  0.15792942, -0.12296642, ...,  0.02230344,
         0.06455814, -0.00190917],
       [-0.32610077, -0.28527686, -0.21016292, ..., -0.0082978 ,
         0.03415923,  0.03400658],
       [-0.09342935, -0.48524377, -0.25819147, ...,  0.04593074,
        -0.03669815,  0.02808005]])

In [29]:
for i in range(len(labels_product)):
    labels_product[i]=np.array(labels_product[i])

In [30]:
# list of numpy array into single numpy array
labels_product=np.stack(labels_product, axis=0)

In [31]:
labels_product.shape

(6040, 50)

In [32]:
#create train and validation set
indices = np.arange(data.shape[0]) 
np.random.shuffle(indices) 
data = data[indices]
labels_product = labels_product[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels_product[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels_product[-num_validation_samples:]

In [33]:
y_train.shape

(4832, 50)

In [34]:
# create dictonary for product embedding
embeddings_index=dict(zip(data_new["asin"],data_new["embedding"]))

In [35]:
type(embeddings_index["B00002N7HY"])

numpy.ndarray

In [36]:
# create an embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word.upper())
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [48]:
embedding_matrix.shape

(5014, 50)

In [37]:
# create model 
inputs = Input(shape=(None,), dtype="int32")
# Embed each integer in a 128-dimensional vector
x = Embedding(embedding_matrix.shape[0], 50,embeddings_initializer=Constant(embedding_matrix),trainable=True)(inputs)
# Add 2 bidirectional LSTMs
x, h1, c1, h2, c2 = Bidirectional(LSTM(128, return_sequences=True, return_state=True))(x)
x, h1, c1, h2, c2 = Bidirectional(LSTM(128, return_sequences=True, return_state=True))(x, initial_state=[h1, c1, h2, c2])
h = Concatenate()([h1, h2])
outputs = Dense(y_train.shape[1], activation="relu")(h)
model = Model(inputs, outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 50)     250700      input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   [(None, None, 256),  183296      embedding[0][0]                  
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) [(None, None, 256),  394240      bidirectional[0][0]              
                                                                 bidirectional[0][1]          

In [38]:
model1=model

In [39]:
# Compiling with selfdefined loss which is Cosine Similarity
import tensorflow as tf
model1.compile("adam",metrics=["mse"],loss=tf.keras.losses.CosineSimilarity(axis=1))

In [40]:
model1.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_val, y_val))

Epoch 1/10
151/151 [==============================] - 16s 108ms/step - loss: -0.1072 - mse: 0.0816 - val_loss: -0.1219 - val_mse: 0.1005
Epoch 2/10
151/151 [==============================] - 15s 97ms/step - loss: -0.1284 - mse: 0.0964 - val_loss: -0.1197 - val_mse: 0.1067
Epoch 3/10
151/151 [==============================] - 15s 97ms/step - loss: -0.1532 - mse: 0.1353 - val_loss: -0.1265 - val_mse: 0.1152
Epoch 4/10
151/151 [==============================] - 14s 91ms/step - loss: -0.1742 - mse: 0.0898 - val_loss: -0.1381 - val_mse: 0.1080
Epoch 5/10
151/151 [==============================] - 13s 87ms/step - loss: -0.1881 - mse: 0.0881 - val_loss: -0.1394 - val_mse: 0.0997
Epoch 6/10
151/151 [==============================] - 15s 99ms/step - loss: -0.2031 - mse: 0.0858 - val_loss: -0.1432 - val_mse: 0.0665
Epoch 7/10
151/151 [==============================] - 15s 97ms/step - loss: -0.2124 - mse: 0.0735 - val_loss: -0.1394 - val_mse: 0.0965
Epoch 8/10
151/151 [===========================

In [41]:
#saving the trained cosine similarity model
model_json = model1.to_json()
with open("lstm_model_regression_cosine.json", "w") as json_file:
    json_file.write(model_json)

In [45]:
layer_name = 'concatenate'
intermediate_layer_model = Model(inputs=model1.input,
                                 outputs=model1.get_layer(layer_name).output)
intermediate_output2 = intermediate_layer_model.predict(data)

In [68]:
intermediate_output2.shape

(6040, 256)

In [47]:
data_new.shape

(6040, 18)

In [84]:
embed=pd.DataFrame(intermediate_output2)

In [101]:
#adding final embeddings to product data for search use
data_new['final embeddings']= embed.values.tolist()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [102]:
data_new.head()

,category,description,title,also_buy,brand,rank,also_view,main_cat,price,asin,tech1,image,date,feature,similar_item,details,tech2,embedding,final embeddings
0,"['Appliances', 'Parts & Accessories', 'Range P...",Outfit your dryer with top-notch connections w...,"Leviton 5050 B01-0-000 Electrical Receptacle, ...","B000FP8LK6, B00002N7IL, B000FPDO4Y, B00NPZH820...",Leviton,">#155,450 in Tools & Home Improvement (See top...",NaN,Tools & Home Improvement,$6.98,B00002N7HY,"{'Part Number': 'B01-05050-000', 'Item Weight'...",NaN,"November 9, 1999","['N/A', 'Made in USA', 'Built of durable therm...","[{'asin': '', 'href': '', 'name': 'Leviton 505...",NaN,NaN,"[1.527734398841858, -0.3872746229171753, 0.390...","[0.04236961156129837, 0.04030364751815796, -0...."
1,"['Appliances', 'Parts & Accessories', 'Range P...",Outfit your dryer with top-notch connections w...,"Leviton 5206 50 Amp, 125/250 Volt, NEMA 10-50R...","B000FP8LK6, B00002N7HY, B000GAQG5C, B004LYLP7I...",Leviton,">#411,187 in Tools & Home Improvement (See top...","['B00002N7HY', 'B074Q2KZGV', 'B000FPCL2K', 'B0...",Tools & Home Improvement,NaN,B00002N7IL,"{'Part Number': 'B01-5206-000', 'Item Weight':...",NaN,"July 7, 2004","['Fits all wallplates with 2.15"" diameter cent...","[{'asin': '', 'href': '', 'name': 'Leviton 520...",NaN,NaN,"[1.8896135091781616, -0.14123277366161346, 0.4...","[0.03194904699921608, 0.01351519301533699, -0...."
2,"['Appliances', 'Parts & Accessories', 'Dryer P...",White heat keeper kitProFlex heat keeper kit. ...,"Dundas Jafine CHK100ZW CHK100ZW6 Vents, 4-Inch...","B0015UGPWQ, B001AAEG5O, B000BO76WU, B00009W3HD...",Dundas Jafine,">#78,624 in Tools & Home Improvement (See top ...",NaN,Tools & Home Improvement,$9.90,B00004YWK2,"{'Part Number': 'CHK100ZW', 'Item Weight': '10...",NaN,"June 16, 2006","['Directs warm, moist air from electric clothe...","[{'asin': '', 'href': '', 'name': 'Dundas Jafi...",NaN,NaN,"[-0.30327674746513367, -0.3628358840942383, -0...","[-0.04139666259288788, 0.1698998212814331, 0.5..."
3,"['Appliances', 'Parts & Accessories']",Keep your air humidifier operating at peak eff...,"Holmes ""C"" Humidifier Filter, HWF65PDQ-U","B0002TSA8I, B0002GD194, B0175Y17SW, B01G6921ZU",Holmes,">#1,235 in Home & Kitchen (See Top 100 in Home...",NaN,Amazon Home,$5.98,B00005OU6T,NaN,['https://images-na.ssl-images-amazon.com/imag...,NaN,"['Improves efficiency of the humidifier', 'Ext...","[{'asin': '', 'href': '', 'name': 'Holmes ""C"" ...",NaN,NaN,"[2.198042631149292, -0.004325718153268099, 0.3...","[-0.03386007621884346, 0.09112260490655899, 0...."
4,"['Appliances', 'Parts & Accessories']","Holmes ""A"" Humidifier Filter, HWF62","Holmes ""A"" Humidifier Filter, HWF62",B0002TSA8I,Holmes,">#1,074 in Home & Kitchen (See Top 100 in Home...",NaN,Amazon Home,$11.89,B00006IV17,NaN,['https://images-na.ssl-images-amazon.com/imag...,NaN,"['Nullify', 'Imported', 'Fits Holmes, Sunbeam,...","[{'asin': '', 'href': '', 'name': 'Holmes ""A"" ...",NaN,NaN,"[-1.6430426836013794, 0.4439624845981598, 0.28...","[0.021733354777097702, 0.024240538477897644, -..."


In [60]:
data_new["final embeddings"][6000]

'0.2033154,-0.057392657,0.35937566,0.11636543,-0.42625442,0.50118834,-0.03654896,0.018049056,-0.21843931,-0.04140938,0.050520767,0.112141185,0.084609225,0.30546802,0.033074964,-0.021906747,0.65713423,-0.036231417,-0.10690814,-0.067608505,-0.51340353,-0.17250134,0.16784346,0.23783922,-0.038251396,-0.1297391,-0.20096295,-0.7994098,0.07475571,0.097155936,0.23007153,-0.07012734,0.09719781,-0.19203652,0.20097062,-0.06664459,0.437079,-0.10941019,0.20576228,-0.01030381,0.15700608,0.3081457,-0.7423693,-0.06647701,-0.057752773,0.07704966,-0.04723751,-0.02231735,-0.04765136,0.018636541,0.017052278,0.19553831,0.1960524,-0.106479675,-0.14802507,-0.12599117,0.010080068,0.0059047416,-0.001235041,-0.023018802,-0.26198596,-0.37511703,0.62930346,-0.55735254,0.14985278,0.011337207,-0.0386385,0.10991068,-0.018432489,0.10426997,-0.6608119,-0.0861401,0.006030084,-0.66279614,0.008123448,-0.17445849,0.015066386,0.09972785,-0.05294113,0.04840211,0.07961507,0.011308562,-0.09596592,-0.042309176,-0.6921061,-0.01

In [92]:
#saving the datadrame which has embeddings with product information
with open("data_new.pkl","wb") as f:
    pickle.dump(data_new,f)